In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,55,2024-02-14,-1,121.740741,100.787037,0.542222,10.844444,27.681481,0.220278,119.859259,...,11.860377,22.028302,0.169000,110.720755,0,2,2,2,1,2024
1,55,2024-02-14,-3,114.759259,96.090741,0.542259,11.016667,29.288889,0.204370,119.524074,...,12.401852,26.046296,0.229259,113.912963,0,2,2,2,20,2024
2,55,2024-02-14,1,110.611111,96.157407,0.535407,11.229630,22.235185,0.222000,114.820370,...,9.958491,25.258491,0.249774,120.467925,0,2,2,1,16,2024
3,55,2024-02-14,1,111.925926,95.446296,0.534907,10.640741,24.385185,0.188315,115.570370,...,11.605769,25.101923,0.182654,117.596154,0,2,2,2,4,2024
4,56,2024-02-14,-1,123.654545,101.570909,0.584436,11.389091,23.649091,0.179673,121.749091,...,11.442593,25.151852,0.186963,115.037037,0,2,2,2,12,2024
5,54,2024-02-14,-1,113.867925,97.109434,0.539830,10.558491,25.211321,0.178472,116.524528,...,10.659259,24.342593,0.198222,122.301852,0,2,2,1,3,2024
6,54,2024-02-14,-7,114.566038,102.571698,0.540453,11.762264,19.692453,0.177283,111.656604,...,11.400000,25.631481,0.213741,118.568519,0,2,2,2,30,2024
7,54,2024-02-14,1,113.339623,98.220755,0.530528,11.049057,24.177358,0.205245,114.326415,...,12.529630,21.831481,0.178500,108.472222,0,2,2,2,11,2024
8,55,2024-02-14,1,112.166667,100.922222,0.530074,12.581481,22.498148,0.175000,110.827778,...,10.664815,22.279630,0.203833,119.018519,0,2,2,2,27,2024
9,54,2024-02-14,-2,118.716981,99.369811,0.565208,11.284906,23.252830,0.177321,118.247170,...,10.690741,26.042593,0.178537,119.318519,0,2,2,1,26,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', show=True):
    if model_name == 'FCNN':
        fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
        fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)
        
        model_file = fcnns.iloc[0]['File']
        
        model = torch.load('Models/'+model_file)
        scaler = joblib.load('Models/Scalers/FCNN.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'FCNN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-14,ATL,CHO,1.400000,3.050000,1
1,2024-02-14,NYK,ORL,2.360000,1.617284,0
2,2024-02-14,MIA,PHI,2.400000,1.602410,0
3,2024-02-14,CHI,CLE,4.000000,1.263158,0
4,2024-02-14,IND,TOR,1.602410,2.400000,1
5,2024-02-14,BRK,BOS,7.500000,1.100000,0
6,2024-02-14,WAS,NOP,7.250000,1.105263,0
7,2024-02-14,HOU,MEM,1.740741,2.200000,1
8,2024-02-14,SAS,DAL,5.550000,1.160000,0
9,2024-02-14,SAC,DEN,2.700000,1.487805,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-14,ATL,CHO,1.400000,3.050000,1
1,2024-02-14,NYK,ORL,2.360000,1.617284,0
2,2024-02-14,MIA,PHI,2.400000,1.602410,0
3,2024-02-14,CHI,CLE,4.000000,1.263158,0
4,2024-02-14,IND,TOR,1.602410,2.400000,1
5,2024-02-14,BRK,BOS,7.500000,1.100000,0
6,2024-02-14,WAS,NOP,7.250000,1.105263,0
7,2024-02-14,HOU,MEM,1.740741,2.200000,1
8,2024-02-14,SAS,DAL,5.550000,1.160000,0
9,2024-02-14,SAC,DEN,2.700000,1.487805,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-14,ATL,CHO,1.400000,3.050000,1
1,2024-02-14,NYK,ORL,2.360000,1.617284,0
2,2024-02-14,MIA,PHI,2.400000,1.602410,0
3,2024-02-14,CHI,CLE,4.000000,1.263158,0
4,2024-02-14,IND,TOR,1.602410,2.400000,1
5,2024-02-14,BRK,BOS,7.500000,1.100000,0
6,2024-02-14,WAS,NOP,7.250000,1.105263,0
7,2024-02-14,HOU,MEM,1.740741,2.200000,1
8,2024-02-14,SAS,DAL,5.550000,1.160000,0
9,2024-02-14,SAC,DEN,2.700000,1.487805,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-14,ATL,CHO,1.400000,3.050000,1
1,2024-02-14,NYK,ORL,2.360000,1.617284,1
2,2024-02-14,MIA,PHI,2.400000,1.602410,0
3,2024-02-14,CHI,CLE,4.000000,1.263158,0
4,2024-02-14,IND,TOR,1.602410,2.400000,1
5,2024-02-14,BRK,BOS,7.500000,1.100000,0
6,2024-02-14,WAS,NOP,7.250000,1.105263,0
7,2024-02-14,HOU,MEM,1.740741,2.200000,1
8,2024-02-14,SAS,DAL,5.550000,1.160000,0
9,2024-02-14,SAC,DEN,2.700000,1.487805,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-14,ATL,CHO,1.400000,3.050000,1.0
1,2024-02-14,NYK,ORL,2.360000,1.617284,1.0
2,2024-02-14,MIA,PHI,2.400000,1.602410,0.0
3,2024-02-14,CHI,CLE,4.000000,1.263158,0.0
4,2024-02-14,IND,TOR,1.602410,2.400000,1.0
5,2024-02-14,BRK,BOS,7.500000,1.100000,0.0
6,2024-02-14,WAS,NOP,7.250000,1.105263,0.0
7,2024-02-14,HOU,MEM,1.740741,2.200000,1.0
8,2024-02-14,SAS,DAL,5.550000,1.160000,0.0
9,2024-02-14,SAC,DEN,2.700000,1.487805,0.0
